In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

Carregar os dados
data = pd.read_csv('data.csv')

Separar os dados em features (X) e rótulos (y)
X = data.drop('nome', axis=1).values
y = data['nome'].values

Converter os rótulos categóricos para valores numéricos
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

Dividir os dados em conjunto de treinamento e conjunto de teste
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

Transformar os rótulos em codificação one-hot
y_train_onehot = tf.keras.utils.to_categorical(y_train_encoded)
y_test_onehot = tf.keras.utils.to_categorical(y_test_encoded)


model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(y_train_onehot.shape[1], activation='softmax')
])


Compilar o modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Treinar o modelo
model.fit(X_train, y_train_onehot, epochs=200, batch_size=64, validation_data=(X_test, y_test_onehot))

Prever rótulos para o conjunto de teste
y_pred_onehot = model.predict(X_test)
y_pred_encoded = np.argmax(y_pred_onehot, axis=1)
y_pred = label_encoder.inverse_transform(y_pred_encoded)

Criar matriz de confusão
cm = confusion_matrix(label_encoder.inverse_transform(y_test_encoded), y_pred)

Visualizar a matriz de confusão
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d', xticklabels=labelencoder.classes, yticklabels=labelencoder.classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()